In [1]:
FILTERD_BOOKS = r'..\..\data\processed\For content base\filtired_books_cb.pkl'
SIM_MATRIX = r'..\..\data\processed\For content base\similarity_matrix_cb.pkl'

# importing


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import ast
import nltk
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, cosine_distances
from sklearn.preprocessing import StandardScaler

import joblib
import requests

In [4]:
books_df = pd.read_csv(FILTERD_BOOKS)
sim_martix = joblib.load(SIM_MATRIX)

In [5]:
books_df

,ISBN,Title,Author,Publication_year,Publisher,Avg_rating,Rating_count,Book_image
0,1567407781,The Witchfinder (Amos Walker Mystery Series),Loren D. Estleman,1998,Brilliance Audio - Trade,6.000000,1,http://images.amazon.com/images/P/1567407781.0...
1,8445071408,El Senor De Los Anillos: LA Comunidad Del Anil...,J. R. R. Tolkien,2001,Minotauro,6.562500,16,http://images.amazon.com/images/P/8445071408.0...
2,8445071769,El Senor De Los Anillos: Las DOS Torres (Lord ...,J. R. R. Tolkien,2001,Minotauro,6.000000,10,http://images.amazon.com/images/P/8445071769.0...
3,8445071777,El Senor De Los Anillos: El Retorno Del Rey (T...,J. R. R. Tolkien,2001,Distribooks,6.500000,10,http://images.amazon.com/images/P/8445071777.0...
4,3257203659,Der illustrierte Mann. ErzÃ?Â¤hlungen.,Ray Bradbury,2002,Diogenes,7.666667,3,http://images.amazon.com/images/P/3257203659.0...
...,...,...,...,...,...,...,...,...
9010,0439230209,Raccoons on the Roof (Animal Ark),Ben M. Baglio,2001,Scholastic,7.000000,1,http://images.amazon.com/images/P/0439230209.0...
9011,0060093269,Two of a Kind #30: Making a Splash (Two of a K...,Mary-Kate &amp; Ashley Olsen,2003,HarperEntertainment,9.000000,1,http://images.amazon.com/images/P/0060093269.0...
9012,3404204433,Am Ende des Regenbogens.,Alan Dean Foster,2002,LÃ?Â¼bbe,8.000000,1,http://images.amazon.com/images/P/3404204433.0...
9013,8420614556,Lewis Carroll: A Traves Del Espejo Y Lo Que Al...,Lewis Carroll,1986,Lectorum Pubns (Adult),7.000000,1,http://images.amazon.com/images/P/8420614556.0...


In [6]:
sim_martix.head()

ISBN,1567407781,8445071408,8445071769,8445071777,3257203659,3257207522,3257208634,3257212429,3257214154,3257216416,...,8484504794,9500416689,0307129683,0395647398,5550534274,0439230209,0060093269,3404204433,8420614556,0440400988
Title,,,,,,,,,,,,,,,,,,,,,
The Witchfinder (Amos Walker Mystery Series),1.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
El Senor De Los Anillos: LA Comunidad Del Anillo (Lord of the Rings (Spanish)),0.0,1.000000,1.000000,0.666667,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.333333,0.0,0.333333,0.0,0.333333,0.0,0.000000,0.0,0.0
El Senor De Los Anillos: Las DOS Torres (Lord of the Rings (Paperback)),0.0,1.000000,1.000000,0.666667,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.333333,0.0,0.333333,0.0,0.333333,0.0,0.000000,0.0,0.0
"El Senor De Los Anillos: El Retorno Del Rey (Tolkien, J. R. R. Lord of the Rings. 3.)",0.0,0.666667,0.666667,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.333333,0.0,0.333333,0.0,0.333333,0.0,0.000000,0.0,0.0
Der illustrierte Mann. ErzÃ?Â¤hlungen.,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.333333,0.333333,0.333333,0.333333,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.333333,0.0,0.0


---

# Sort popular books


In [7]:
books_df.head(1)

,ISBN,Title,Author,Publication_year,Publisher,Avg_rating,Rating_count,Book_image
0,1567407781,The Witchfinder (Amos Walker Mystery Series),Loren D. Estleman,1998,Brilliance Audio - Trade,6.0,1,http://images.amazon.com/images/P/1567407781.0...


In [8]:
def get_top_n_books(df, n):
    return df.sort_values(by=['Rating_count', 'Avg_rating'], ascending=False).head(n)

In [9]:
get_top_n_books(books_df, 20)

,ISBN,Title,Author,Publication_year,Publisher,Avg_rating,Rating_count,Book_image
177,043935806X,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,Scholastic,5.571856,334,http://images.amazon.com/images/P/043935806X.0...
202,0439136369,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,2001,Scholastic,5.345133,226,http://images.amazon.com/images/P/0439136369.0...
127,0439136350,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,1999,Scholastic,6.467005,197,http://images.amazon.com/images/P/0439136350.0...
75,0812550706,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,1994,Tor Books,5.302564,195,http://images.amazon.com/images/P/0812550706.0...
175,0439139597,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2000,Scholastic,6.541237,194,http://images.amazon.com/images/P/0439139597.0...
214,0439139600,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2002,Scholastic Paperbacks,5.103627,193,http://images.amazon.com/images/P/0439139600.0...
176,0439064864,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,1999,Scholastic,6.611765,170,http://images.amazon.com/images/P/0439064864.0...
95,0590353403,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,1998,Scholastic,6.363095,168,http://images.amazon.com/images/P/0590353403.0...
93,0618002219,The Hobbit: or There and Back Again,J.R.R. Tolkien,1999,Houghton Mifflin Company,5.014286,70,http://images.amazon.com/images/P/0618002219.0...
383,0618002227,The Fellowship of the Ring (The Lord of the Ri...,J. R. R. Tolkien,1999,Houghton Mifflin Company,6.206349,63,http://images.amazon.com/images/P/0618002227.0...


---


# Recommendation fun


In [11]:
sim_martix.head(1)

ISBN,1567407781,8445071408,8445071769,8445071777,3257203659,3257207522,3257208634,3257212429,3257214154,3257216416,...,8484504794,9500416689,0307129683,0395647398,5550534274,0439230209,0060093269,3404204433,8420614556,0440400988
Title,,,,,,,,,,,,,,,,,,,,,
The Witchfinder (Amos Walker Mystery Series),1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
recomm_books = pd.DataFrame(sim_martix['1567407781'].sort_values(
    ascending=False).head(10)).reset_index()[1:]

In [15]:
recomm_books

,Title,1567407781
1,Jitterbug: A Novel of Detroit,0.666667
2,The Witchfinder (Bookcassette(r) Edition),0.666667
3,Blood Work (Nova Audiobooks),0.666667
4,The Ninth Garfield Fat Cat 3-Pack (Garfield Fa...,0.333333
5,Arthur and the Popularity Test : A Marc Brown ...,0.333333
6,For Kicks,0.333333
7,The Transall Saga,0.333333
8,How to Study the Bible for Yourself,0.333333
9,The Picture of Dorian Gray (Oxford World's Cla...,0.333333


In [19]:
def recommmendation_by_id(movie_id):
    recomm_movies = pd.DataFrame(sim_martix[movie_id].sort_values(
        ascending=True).head(10)).reset_index()[1:]
    recom_df = pd.DataFrame(columns=books_df.columns)
    for title in recomm_movies['Title']:
        recom = books_df[books_df['Title'] == title]
        recom_df = pd.concat([recom_df, recom])
    return recom_df

In [20]:
recommmendation_by_id('1567407781')

,ISBN,Title,Author,Publication_year,Publisher,Avg_rating,Rating_count,Book_image
5909,0553251724,Gambit,Rex Stout,1985,Bantam Books,5.6,5,http://images.amazon.com/images/P/0553251724.0...
5908,1586215809,The Big Bad Wolf: A Novel,James Patterson,2003,Time Warner Audiobooks,8.0,1,http://images.amazon.com/images/P/1586215809.0...
5907,0743506642,Stalker (Peter Decker &amp; Rina Lazarus Novel...,Faye Kellerman,2000,Simon &amp; Schuster Audio,7.0,1,http://images.amazon.com/images/P/0743506642.0...
5100,0375728198,A Bend in the Road,Nicholas Sparks,2002,Random House Large Print Publishing,7.0,1,http://images.amazon.com/images/P/0375728198.0...
5906,1586211781,A Bend in the Road,Nicholas Sparks,2001,Time Warner Audio Major,10.0,1,http://images.amazon.com/images/P/1586211781.0...
5905,1570429693,Wish You Well,David Baldacci,2000,Time Warner Audio Major,10.0,1,http://images.amazon.com/images/P/1570429693.0...
5928,1570429685,Wish You Well,David Baldacci,2000,Time Warner Audio Major,9.5,4,http://images.amazon.com/images/P/1570429685.0...
5904,0312932812,Crazy for You (Jennifer Crusie 2004),Jennifer Crusie,2004,St. Martin's Press,10.0,2,http://images.amazon.com/images/P/0312932812.0...
5903,1572702516,And Then There Were None (Audio Editions Myste...,Agatha Christie,2001,Audio Partners,10.0,1,http://images.amazon.com/images/P/1572702516.0...
5902,1570429227,Roses Are Red,James Patterson,2000,Time Warner Audio Major,7.0,1,http://images.amazon.com/images/P/1570429227.0...
